In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from scipy.stats import chisquare

In [ ]:
#Font sizes for figures 
label_fontsize = 15
title_fontsize = 17
tick_size=13

In [ ]:

etm_df = pd.read_csv('H:/data/Results/ETM_resultater_redcap.csv')
etm_df = etm_df.loc[etm_df['voxel_ktrans'] > 0]
etm_df = etm_df[['record_id', 'average_ktrans', 'average_ve', 'average_vp', 'average_kep', 'voxel_ktrans', 'voxel_ve', 'voxel_vp', 'voxel_kep', 'mask', 'average_tumor_volume']]
etm_df['record_id'] = etm_df['record_id'].str.lstrip('GYN ')
etm_df['record_id'] = etm_df['record_id'].str.lstrip('0')
avrg_kt_array = (etm_df['average_ktrans'].to_numpy())*60
voxel_kt_array = (etm_df['voxel_ktrans'].to_numpy())*60
tumor_vol = etm_df['average_tumor_volume'].to_numpy()
avrg_kep_array = (etm_df['average_kep'].to_numpy())*60
avrg_ve_array = etm_df['average_ve'].to_numpy()
avrg_vp_array = etm_df['average_vp'].to_numpy()
voxel_kep_array = (etm_df['voxel_kep'].to_numpy())*60
voxel_ve_array = etm_df['voxel_ve'].to_numpy()
voxel_vp_array = etm_df['voxel_vp'].to_numpy()
etm_df = etm_df.astype(str)
etm_df

In [ ]:
#Kolmogorov-Smirnov to test if the data is normally distributed 
ks_result = kstest(avrg_kt_array, cdf = 'norm')
if ks_result[1] > 0.05: #t-test if the data is normally distributed 
    print(f"K-S statistic: {ks_result[0]}")
    print(f"p-value: {ks_result[1]}")
    t_test = stats.ttest_rel(avrg_kt_array, voxel_kt_array)
    print(t_test)
else: 
    print('Data is not normally distributed')
    print(f"K-S statistic: {ks_result[0]}")
    print(f"p-value: {ks_result[1]}")
    #mann_whitney = mannwhitneyu(np.array(Kt_avrg), np.array(Kt_voxel))
    #print(mann_whitney)
    res  = wilcoxon(avrg_kt_array, voxel_kt_array)
    print('Wilcoxon p-value:', res.pvalue)
    print('Wilcoxon z-statistic:', res.statistic)

In [ ]:
#Pearson correlation 
pearson = stats.pearsonr(avrg_kt_array, voxel_kt_array)
print(pearson)

#Spearman correlation 
spearman = spearmanr(avrg_kt_array, voxel_kt_array)
print(spearman)

In [ ]:
print(np.median(avrg_kt_array))
print(np.median(voxel_kt_array))

In [ ]:

#Plot Kt difference vs. tumor volume
diff = abs(avrg_kt_array-voxel_kt_array)
fig = plt.figure()
plt.rcParams.update({'font.family':'Times New Roman'})
plt.scatter(tumor_vol, diff, color='tab:blue')
plt.xticks(size=tick_size)
plt.yticks(size=tick_size)
plt.xlabel('Tumor volume [ml]', size=label_fontsize)
plt.ylabel('Absolute difference $[min^{-1}]$', size=label_fontsize)
plt.title('$K^{trans}$ difference vs. tumor volume', size=title_fontsize)
#plt.ylim(0,0.05)
plt.grid()
fig.savefig('H:/data/Results/rPACS/ktrans_vs_tumorvol_rPACS.eps', bbox_inches='tight')

In [ ]:
#kt Bland-Altman plot
fig = plt.figure()
bland_altman_plot(avrg_kt_array, voxel_kt_array, 1.11)
plt.xticks(size=tick_size)
plt.yticks(size=tick_size)
plt.title('Bland-Altman plot', size=17)
fig.savefig('H:/data/Results/rPACS/Bland_altman_rPACS.eps', bbox_inches='tight')

In [ ]:
#Clinical patient data 
hist_df_og = pd.read_csv('H:/data/master endometrial data/patients.csv')
hist_df = hist_df_og[['Subj', 'TimeFollowUp', 'StatusFollowUp', 'HistGrade2G']]
hist_df = hist_df.dropna()
hist_df.rename(columns = {'Subj':'record_id'}, inplace = True)
hist_df = hist_df.astype(str)


In [ ]:
#Average modeling results 
result_df = pd.merge(etm_df, hist_df, how='inner', on='record_id')
result_Kt = ((result_df['average_ktrans'].astype(float)).to_numpy())*60
result_kep = ((result_df['average_kep'].astype(float)).to_numpy())*60
result_vp = ((result_df['average_vp'].astype(float)).to_numpy())
result_ve = ((result_df['average_ve'].astype(float)).to_numpy())
histgrade = ((result_df['HistGrade2G'].astype(float)).to_numpy())

In [ ]:
#Voxelwise modeling results 
result_Kt = ((result_df['voxel_ktrans'].astype(float)).to_numpy())*60
result_kep = ((result_df['voxel_kep'].astype(float)).to_numpy())*60
result_vp = ((result_df['voxel_vp'].astype(float)).to_numpy())
result_ve = ((result_df['voxel_ve'].astype(float)).to_numpy())
print(result_vp)

In [ ]:
#Split Ktrans by median 
median = np.median(result_Kt)

observed_kt = []
for i in result_Kt: 
    if i < median: 
        observed_kt.append(1)
    elif i > median: 
        observed_kt.append(0)

In [ ]:
expected_lower = observed_kt.count(0) #Denne verdien er 30
observed_lower =  histgrade.size - np.count_nonzero(histgrade==1) #Denne verdien er 40

expected_upper = observed_kt.count(1) #Denne verdien er 30
observed_upper = np.count_nonzero(histgrade == 1) #Denne verdien er 20

#Chi-square test
chisquare_res = chisquare([[observed_lower,observed_upper], [expected_lower, expected_upper]]) # chisquare([[40,20], [30,30]])
print(chisquare_res)

In [ ]:
#Plot ROC curve 
fpr, tpr, _ = roc_curve(histgrade, result_Kt)
auc_score = roc_auc_score(histgrade, result_Kt)
print(fpr)
print(tpr)
print(auc_score)
plt.rcParams['font.family'] = 'Times New Roman'

label_fontsize = 15
title_fontsize = 17
tick_size=13

plt.plot(fpr, tpr, 'tab:blue', label = 'AUC = 0.536' )
plt.plot([0, 1], [0, 1], "k--")
plt.legend(fontsize=label_fontsize)
plt.xticks(size=tick_size)
plt.yticks(size=tick_size)
plt.ylabel('True Positive Rate', size=label_fontsize)
plt.xlabel('False Positive Rate', size=label_fontsize)
plt.title('$K^{trans}$ vs. histologic tumor grade', size=title_fontsize)
plt.savefig('H:/data/Results/ROC_kt_histgrade.eps', bbox_inches = 'tight')
plt.show()

In [ ]:
#Split by high-risk and low-risk tumor grade 
high_grade = result_df.loc[result_df['HistGrade2G'] == '1']
low_grade = result_df.loc[result_df['HistGrade2G'] == '0']